In [5]:
#Imports and paths setup
import sys
import rp
import torch
import numpy as np
from einops import rearrange
import shlex


top_dir = rp.get_git_toplevel()

ltx_dir = rp.path_join(top_dir, 'LTX2')
nfs_models_dir = rp.path_join(ltx_dir, 'models')

sys.path+=[
    nfs_models_dir
]

from download_models import local_download_dir, download_from_web
local_models_dir = download_models.local_download_dir

models_dir = local_models_dir

print(f"Running in notebook: {IN_NOTEBOOK}")
print(f"Top directory: {top_dir}")

Running in notebook: True
Top directory: /root/CleanCode/Experiments/LTX_Tests


In [ ]:
IN_NOTEBOOK = rp.running_in_jupyter_notebook()

In [ ]:
models_dir = rp.path_join(top_dir, 'LTX2', 'models')
vae_path = rp.path_join(models_dir, 'ltx-2-19b-distilled.safetensors')
device = rp.select_torch_device(prefer_used=True, reserve=True)
dtype = torch.bfloat16

print(f"Device: {device}, dtype: {dtype}")
print("Loading VAE...")
vae_encoder = load_video_vae_encoder(vae_path, device=device, dtype=dtype)
vae_decoder = load_video_vae_decoder(vae_path, device=device, dtype=dtype)
print("VAE loaded!")

rp._run_sys_command(f"rclone {shlex.quote()}")